In [ ]:
!pip install pyspark
import sys
 
from pyspark import SparkContext, SparkConf
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 20.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=6007684b5bbf2c59b7986be25c514258ce1c7583602deacdc4eea1cd30d32c29
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
Mounted at /content/drive


In [ ]:
import math
import re
################################# functions#######################################
####### remove stopwords #####
def removeStopWords(line):
  words = [word for word in line.split(" ") if word.lower() not in stopwordsSet]
  new_text = " ".join(words)
  return new_text

####### remove URLS #####
def removeUrl(line):
  return re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", "", line)

####### remove Punctuation #####
def removePunctuation(line):
  words = line.split(" ")
  removedWords = []
  for word in words:
    word2 = re.sub(r'[^\w\s\n<docid></docid><text></text>]', '', word)
    if word2 != '':
            removedWords.append(word2)
  new_text = " ".join(removedWords)
  return new_text

####### remove Tags #####
def removeTags(line):
  return re.compile(r'<(?!docid|text|/docid|/text).*?>').sub('',line)

#get words and their docids 
def wordDocId(input):
  str = input.split("<docid>")
  data = []
  for a in str:
    if a:
      documentno = a.split("</docid>")[0]
      # data.append(documentno)
      texthere = a.split("</docid>")[1].replace('<text>','').replace('</text>','')
      # data.append(texthere)
      words = texthere.split(" ")
      for word in words:
        str = word,documentno
        data.append(str)
  return data

############ create term frequency matrix ##################### 
def createTermFreqMatrix(line):
  ((word,docId),freq) = line
  freq = 1+ math.log10(freq) # log weighted frequency
  return (word,(docId,freq))

#create document frequency
def calculateDocFrequency(line):
  (word,dataArray) = line
  inverseFreq = math.log10(numberOfDocuments/1+len(dataArray))
  return (word,inverseFreq)

######################## TF-IDF ##############################
def tfidf(line):
  (word,(dataArray,docFreq)) = line  ##ex:[('siege', ([('10000', 1.6020599913279625)], 0.6989700043360189)),...
  newDataArray = []
  for data in dataArray:
    (docid,termFreq) = data
    tfIdfFreq = termFreq*docFreq #TF-IDF 
    newData = (docid,tfIdfFreq)
    newDataArray.append(newData)
  return (word,newDataArray)

####### refactor RDD ######
def refactorRDD(array):
  return array

####### euclidean length of each document ######
def euclideanLength(line):
  (docid,array) = line
  sum = 0
  for data in array:
    sum += (data*data) 
  return (docid,math.sqrt(sum))

#write in the format of word@doc-1#freq+doc-2#freq+doc-3#freq+...
def customWriteToFile(line):
 (word,matrixArray) = line
 data = []
 var = word + "@"
 for terms in matrixArray:
   (docId,freq) = terms
   var += str(docId)+"#"+str(freq)+"+"
 return var

############## make the tf cosine normalized ######################
def cosineNorm(line):   # [('siege', [('10000', 1.1197918790850683)]),...
  (word,dataArray) = line
  newDataArray = []
  for data in dataArray:
    (docid,freq) = data
    freq = freq/docEuclidLengthDictionary[docid]
    newTuple = (docid,freq)
    newDataArray.append(newTuple)
  return (word,newDataArray)


In [ ]:
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('InfoRetrieval')
sc = SparkContext.getOrCreate(conf=conf)
file1 = '2.txt' #Enter file name/path here
fileRDD = sc.textFile(file1)
numberOfDocuments = fileRDD.count() #get no: of documents

lowercaseRDD = fileRDD.map(lambda line:line.lower()) # case folding

#remove stopwords
stopwordsSet = set(line.strip() for line in open('stopwords.txt'))
stopWordsRemovedRDD = lowercaseRDD.map(removeStopWords)
# print("check is:",stopWordsRemovedRDD.collect())

#remove url
urlRemovedRDD = stopWordsRemovedRDD.map(removeUrl)

#remove Punctuation
punctRemovedRDD = urlRemovedRDD.map(removePunctuation)

#remove tags
tagsRemovedRDD = punctRemovedRDD.map(removeTags)


#prep RDD to calculate term frequencies
testRDD = tagsRemovedRDD.flatMap(wordDocId) #ex: [('siege', '10000'), ('jerusalem', '10000')....
# print(testRDD.collect())
#calculate same tuples(or words) and combine them together
testRdd2 = testRDD.map(lambda word:(word,1))#ex: [(('siege', '10000'), 1),...
testRdd3 = testRdd2.reduceByKey(lambda a,b: a+b)#ex: [(('siege', '10000'), 4)...

############ create term frequency matrix ##################### 
rdd4 = testRdd3.map(createTermFreqMatrix) #ex: [('siege', ('10000', 1.6020599913279625)),...

#combine all same words together 
freqMatrix = rdd4.groupByKey().mapValues(list)#ex: [('siege', [('10000', 1.6020599913279625)]),...
# print(freqMatrix.collect())
##################################################################

# ######################## Document Frequency ##############################

documentFreq =  freqMatrix.map(calculateDocFrequency) #will have only document frequency of a word
# print(documentFreq.collect()) #ex:[('siege', 0.6989700043360189),...


######################## TF-IDF ##############################

tfidfJoinRdd = freqMatrix.join(documentFreq) #join term frequency matrix to document frequency
# print(tfidfJoinRdd.collect())

tfidfRdd = tfidfJoinRdd.map(tfidf) #calculate TFIDF on joined dataset ex:[('siege', [('10000', 1.1197918790850683)])...
# print(tfidfRdd.collect())

#################### for TFIDF###############################
tempRdd = tfidfRdd.values() #get only docid and tfid values
# print(tempRdd.collect()) #[('10000', 1.1197918790850683)], [('10000', 0.6989700043360189)].....
#################### for TFIDF###############################

rdd3 = tempRdd.flatMap(refactorRDD).groupByKey().mapValues(list) #refactor to get an RDD of (docid,list of termfrequencies in that docid)
# print(rdd3.collect()) #[('10000', [1.1197918790850683, 0.69897000433601...

docEuclidLengthsRdd = rdd3.map(euclideanLength) # get euclidean lengths of each document
# print(docEuclidLengthsRdd.collect()) #[('10000', 4.998304130287607), ('10001', 10.697526447828157)..

############## make the tfidf cosine normalized ######################

docEuclidLengthDictionary = docEuclidLengthsRdd.collectAsMap() #maintain a dictionary for document frequencies for easy reference
# print(docEuclidLengthDictionary)

cosineNormalizedRdd = freqMatrix.map(cosineNorm) #cosine normalize weighted term frequencies
# print(cosineNormalizedRdd.collect())

#  write to file - create CTFIDF_index
cosineNormalizedRdd2 = cosineNormalizedRdd.map(customWriteToFile)
# print(cosineNormalizedRdd2.collect())
cosineNormalizedRdd2.saveAsTextFile('/content/drive/MyDrive/test/CTFIDF_Index')